# Algorithmic Trading

Create a Trading Algorithm for the following (simple) Contrarian Trading Strategy: 

1. At the end of each trading day, you __buy__ the __three worst performing stocks__ among the __30 DJIA__ stocks and hold them for the next trading day.

2. At the end of each trading day, you __short sell__ the __three best performing stocks__ among the __30 DJIA__ stocks and short them for the next trading day.

3. For simplicity reasons, you buy/sell __one share__ of the respective stocks.

4. Underlying idea: The __Market overreacted__ and most recent (extreme) price __trends will revert__ in the near future -> __Contrarian Trading__ (!= Momentum Trading)

5. Develop the code that
- __imports__ required data 
- __measures__ the most recent performance (price change in %)
- __identifies__ the best/worst performing stocks
- __trades__ the stocks with the IBKR API 

In [1]:
import pandas as pd
import yfinance as yf
from ib_insync import *
util.startLoop()

## Get the Symbols

In [2]:
df = pd.read_csv("DJI_Const.csv", header = [0, 1], index_col = 0, parse_dates = [0])
df

Adj Close                                                  \
                  AAPL        AMGN         AXP          BA         CAT   
Date                                                                     
2020-08-31  127.289726  238.217224   98.721237  171.820007  135.369568   
2020-09-01  132.360016  235.903885   99.576393  172.100006  138.927170   
2020-09-02  129.617706  242.731033  101.451889  174.779999  141.990143   
2020-09-03  119.240417  233.129745  101.510201  168.770004  139.602539   
2020-09-04  119.319313  233.590546  102.686020  171.050003  140.953293   
...                ...         ...         ...         ...         ...   
2022-11-09  134.869995  289.649994  145.860001  168.740005  225.539993   
2022-11-10  146.869995  291.010010  154.750000  177.580002  232.449997   
2022-11-11  149.699997  285.019989  154.889999  177.490005  236.490005   
2022-11-14  148.279999  285.299988  154.130005  173.550003  236.500000   
2022-11-15  150.039993  283.600006  153.889999  175.350006  234.589996   

                                                                      ...  \
                   CRM       CSCO         CVX         DIS        DOW  ...   
Date                                                                  ...   
2020-08-31  272.649994  39.405495   76.447708  131.869995  41.147762  ...   
2020-09-01  281.250000  39.228161   75.673492  133.550003  43.108482  ...   
2020-09-02  276.690002  39.592159   75.773689  135.389999  44.886814  ...   
2020-09-03  265.010010  38.220158   74.944817  133.240005  44.129879  ...   
2020-09-04  254.699997  38.098816   74.626007  131.990005  44.321396  ...   
...                ...        ...         ...         ...        ...  ...   
2022-11-09  142.059998  43.910000  177.929993   86.750000  47.680000  ...   
2022-11-10  156.300003  45.660000  181.300003   90.459999  49.959999  ...   
2022-11-11  157.729996  44.790001  186.460007   95.010002  53.139999  ...   
2022-11-14  158.660004  44.740002  186.550003   94.279999  51.950001  ...   
2022-11-15  162.070007  44.900002  188.050003   95.510002  51.599998  ...   

              Volume                                                           \
                MSFT       NKE       PG      TRV      UNH         V        VZ   
Date                                                                            
2020-08-31  28774200   3519800  5596800  2297900  4341600   9326900  14555500   
2020-09-01  25725500   4912400  5671800   969900  2487100   5642900  13358400   
2020-09-02  34080800   6775400  8384800  1391400  2846500   9863800  21711900   
2020-09-03  58400300   7264400  7277900  1299400  3872500  11310200  23126800   
2020-09-04  59664100   5157600  7437700  1460700  2901000  13243600  22250100   
...              ...       ...      ...      ...      ...       ...       ...   
2022-11-09  27852900   6550900  5690700   999200  2672600   7320200  17574600   
2022-11-10  46268000   9361000  8068700  1618000  3426400   8048900  23049000   
2022-11-11  34600900  12595600  7174400  1403300  7032900   9600700  22978900   
2022-11-14  31123300   7134900  8516200  1427300  5238400   5796100  23028500   
2022-11-15  31367300   7241000  7731700  1445900  5102800   7315700  25058900   

                                           
                 WBA       WMT       ^DJI  
Date                                       
2020-08-31   7940100  15078800  517320000  
2020-09-01  10728300  35599400  423410000  
2020-09-02   7672400  17222000  539510000  
2020-09-03   8278000  16005000  650080000  
2020-09-04   6994400  11327400  694640000  
...              ...       ...        ...  
2022-11-09   6830500   5310700  355200000  
2022-11-10   8825000   6770200  480130000  
2022-11-11  13098600   6419700  427790000  
2022-11-14   8301900  12508600  354340000  
2022-11-15   6755300  25504700  390610000  

[558 rows x 186 columns]

In [3]:
symbols = df.Close.columns.to_list()
symbols

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT',
 '^DJI']

In [4]:
symbols.remove("^DJI")

In [5]:
len(symbols)

30

# Get most recent Prices and Performance

In [6]:
yf.Ticker(ticker = "AAPL").get_fast_info() # get_fast_info() NOW!!!

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']

In [7]:
cprice =  yf.Ticker("AAPL").get_fast_info()["last_price"] # updated
cprice

182.88999938964844

In [8]:
last_close = yf.Ticker("AAPL").get_fast_info()["regularMarketPreviousClose"] # updated
last_close

181.82000732421875

In [9]:
perf = cprice / last_close - 1
perf

0.005884897273828127

In [10]:
perf = pd.Series(dtype = float)
perf

Series([], dtype: float64)

In [11]:
symbols

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

In [12]:
count = 1
for symbol in symbols:
    try:
        fast_info = yf.Ticker(ticker = symbol).get_fast_info() # updated
        prc_chg = fast_info["last_price"] / fast_info["regularMarketPreviousClose"] - 1 # updated
        perf.loc[symbol] = prc_chg
        print(count, end = '\r')
        count += 1
    except Exception as e:
        print("{} not found".format(symbol))
print("Download complete.")

Download complete.


In [13]:
perf

AAPL    0.005885
AMGN    0.007930
AXP     0.000196
BA      0.002194
CAT    -0.000426
CRM    -0.001747
CSCO   -0.002447
CVX    -0.013919
DIS    -0.001064
DOW    -0.004739
GS      0.001883
HD      0.003901
HON    -0.010664
IBM    -0.005375
INTC   -0.021924
JNJ    -0.003645
JPM     0.004930
KO     -0.001574
MCD    -0.004392
MMM    -0.008910
MRK     0.003846
MSFT    0.007406
NKE     0.000274
PG     -0.003852
TRV    -0.008412
UNH    -0.002045
V      -0.003514
VZ     -0.004730
WBA    -0.030023
WMT    -0.008150
dtype: float64

In [14]:
perf.sort_values(inplace = True)
perf

WBA    -0.030023
INTC   -0.021924
CVX    -0.013919
HON    -0.010664
MMM    -0.008910
TRV    -0.008412
WMT    -0.008150
IBM    -0.005375
DOW    -0.004739
VZ     -0.004730
MCD    -0.004392
PG     -0.003852
JNJ    -0.003645
V      -0.003514
CSCO   -0.002447
UNH    -0.002045
CRM    -0.001747
KO     -0.001574
DIS    -0.001064
CAT    -0.000426
AXP     0.000196
NKE     0.000274
GS      0.001883
BA      0.002194
MRK     0.003846
HD      0.003901
JPM     0.004930
AAPL    0.005885
MSFT    0.007406
AMGN    0.007930
dtype: float64

In [15]:
perf.index.name = "symbol"
perf 

symbol
WBA    -0.030023
INTC   -0.021924
CVX    -0.013919
HON    -0.010664
MMM    -0.008910
TRV    -0.008412
WMT    -0.008150
IBM    -0.005375
DOW    -0.004739
VZ     -0.004730
MCD    -0.004392
PG     -0.003852
JNJ    -0.003645
V      -0.003514
CSCO   -0.002447
UNH    -0.002045
CRM    -0.001747
KO     -0.001574
DIS    -0.001064
CAT    -0.000426
AXP     0.000196
NKE     0.000274
GS      0.001883
BA      0.002194
MRK     0.003846
HD      0.003901
JPM     0.004930
AAPL    0.005885
MSFT    0.007406
AMGN    0.007930
dtype: float64

# Determine Target Positions

In [16]:
buy_stocks = 3 # buy the 3 worst performing stocks

In [17]:
sell_stocks = 3 # short sell the 3 best performing stocks

In [18]:
shares = 1 # one share per stock

In [19]:
perf.iloc[:buy_stocks] = shares
perf.iloc[-sell_stocks:] = -shares
perf

symbol
WBA     1.000000
INTC    1.000000
CVX     1.000000
HON    -0.010664
MMM    -0.008910
TRV    -0.008412
WMT    -0.008150
IBM    -0.005375
DOW    -0.004739
VZ     -0.004730
MCD    -0.004392
PG     -0.003852
JNJ    -0.003645
V      -0.003514
CSCO   -0.002447
UNH    -0.002045
CRM    -0.001747
KO     -0.001574
DIS    -0.001064
CAT    -0.000426
AXP     0.000196
NKE     0.000274
GS      0.001883
BA      0.002194
MRK     0.003846
HD      0.003901
JPM     0.004930
AAPL   -1.000000
MSFT   -1.000000
AMGN   -1.000000
dtype: float64

In [20]:
target = pd.concat([perf.iloc[:buy_stocks], perf.iloc[-sell_stocks:]]).to_frame().reset_index()
target.columns = ["symbol", "position"]
target

,symbol,position
0,WBA,1.0
1,INTC,1.0
2,CVX,1.0
3,AAPL,-1.0
4,MSFT,-1.0
5,AMGN,-1.0


# Identify Current Positions

In [21]:
ib = IB()

In [22]:
ib.connect()

<IB connected to 127.0.0.1:7497 clientId=1>

In [23]:
pos = ib.positions()
pos

[]

In [29]:
df = util.df(pos)
df

In [ ]:
df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)

In [30]:
if df is not None:
    df["symbol"] = df.contract.apply(lambda x: x.symbol)
    df["conID"] = df.contract.apply(lambda x: x.conId)
else: 
    df = pd.DataFrame(columns = ["symbol", "position"])

In [31]:
df

,symbol,position


## Determine Required Trades (from actual to target positions)

In [32]:
target

,symbol,position
0,WBA,1.0
1,INTC,1.0
2,CVX,1.0
3,AAPL,-1.0
4,MSFT,-1.0
5,AMGN,-1.0


In [33]:
df

,symbol,position


In [34]:
trades = pd.merge(target, df[["symbol", "position"]], "outer", on = "symbol", suffixes = ["_t", "_a"])
trades

,symbol,position_t,position_a
0,WBA,1.0,NaN
1,INTC,1.0,NaN
2,CVX,1.0,NaN
3,AAPL,-1.0,NaN
4,MSFT,-1.0,NaN
5,AMGN,-1.0,NaN


In [35]:
trades.fillna(0, inplace = True)
trades

,symbol,position_t,position_a
0,WBA,1.0,0
1,INTC,1.0,0
2,CVX,1.0,0
3,AAPL,-1.0,0
4,MSFT,-1.0,0
5,AMGN,-1.0,0


In [36]:
trades["trades"] = trades.position_t - trades.position_a 

In [37]:
trades

,symbol,position_t,position_a,trades
0,WBA,1.0,0,1.0
1,INTC,1.0,0,1.0
2,CVX,1.0,0,1.0
3,AAPL,-1.0,0,-1.0
4,MSFT,-1.0,0,-1.0
5,AMGN,-1.0,0,-1.0


In [38]:
trades = trades[trades.trades !=0].set_index("symbol").copy()
trades

,position_t,position_a,trades
symbol,,,
WBA,1.0,0,1.0
INTC,1.0,0,1.0
CVX,1.0,0,1.0
AAPL,-1.0,0,-1.0
MSFT,-1.0,0,-1.0
AMGN,-1.0,0,-1.0


## Execute Trades

In [39]:
# Loop foe each symbol
for symbol in trades.index:
    
    # Fet the trad of the symbol (e.g: -2 is SELL, 1 is BUY, etc)
    to_trade = trades.loc[symbol, "trades"]
   
    # check the trad
    if to_trade > 0: 
        side = "BUY"
    elif to_trade < 0:
        side = "SELL"
   
    # Take the contrast, and subscrice    
    contract = Stock(symbol, "SMART", "USD")
    cds = ib.reqContractDetails(contract)
    
    if len(cds) == 0:
        print("No Contract for {} found.".format(symbol))
   
    # If the is only one CDS    
    elif len(cds) == 1:
        contract = cds[0].contract # take the contract
        order = MarketOrder(side, abs(to_trade)) # create Order
        trade = ib.placeOrder(contract, order) # Place the order!
        while not trade.isDone():
            ib.waitOnUpdate()
        if trade.orderStatus.status == "Filled":
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))
    
    # If the is multiple CDS
    else:
        contract = cds[0].contract
        print("Multiple Contracts for {} found.".format(symbol))
        order = MarketOrder(side, abs(to_trade))
        trade = ib.placeOrder(contract, order)
        while not trade.isDone():
            ib.waitOnUpdate()
        if trade.orderStatus.status == "Filled":
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))  



pos = ib.positions()
df = util.df(pos)
df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
df

BUY WBA @ 21.1
BUY INTC @ 37.98
BUY CVX failed.
SELL AAPL @ 183.01


In [ ]:
target

In [ ]:
ib.disconnect()